In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


In [40]:
import glob


folders = ["data/2019", "data/2020", "data/2021", "data/2022", "data/2023"]


# read all the files in the folders as pandas dataframe
# create an empty list to store the dataframes
dataframes = []
for folder in folders:
    file_name = "geocoded_merged_sales_fixed.csv"
    data = pd.read_csv(f"{folder}/{file_name}")
    if "temp_unique_id" in data.columns:
        # remove that column
        data.drop('temp_unique_id', axis=1, inplace=True)
    if "temp_unique_id_norm" in data.columns:
        data.drop("temp_unique_id_norm", axis=1, inplace=True)
    real_year = folder.split("/")[-1]
    data["year"] = pd.to_datetime(data["sale_date"]).dt.year
    if len(data["year"].unique()) > 1:
        # drop years that are not the real_year
        data = data[data["year"] == int(real_year)]
    print(data["year"].unique(), len(data["year"]))
    print("min ",data["sale_price"].unique().min())
    print("max ",data["sale_price"].unique().max())
    dataframes.append(data)

[2019] 6199
min  143000.0
max  17757750.0
[2020] 5716
min  175000.0
max  34000000.0
[2021] 7928
min  147960.0
max  21750000.0
[2022] 6427
min  167000.0
max  15500000.0
[2023] 4957
min  200000.0
max  23000000.0


In [41]:
# check if each dataframe have only one year
for df in dataframes:
    print(df["year"].unique())

[2019]
[2020]
[2021]
[2022]
[2023]


In [42]:
# Get first dataframe and sort by sale price
df_sample = dataframes[0]
df_sample = df_sample.sort_values('sale_price', ascending=True)
df_sample[["parcel", "sale_price", 'category']].head()

,parcel,sale_price,category
4950,18-01543-008,143000.0,condo
4606,15-00389-006,148900.0,condo
4999,18-06810-092,160000.0,condo
4997,18-06810-130,164000.0,condo
4998,18-06810-090,165000.0,condo


In [43]:
for df in dataframes:
    print(len(df.columns))
    print(df.columns)

28
Index(['sale_date', 'sale_price', 'parcel', 'street_no', 'living_area',
       'category', 'price_per_sf', 'street_name', 'y', 'full_address', 'x',
       'owner_property_count', 'total_room_num', 'half_bathrooms',
       'fireplaces', 'bathrooms', 'exterior_condition', 'year_built',
       'properties', 'interior_condition', 'heat_type', 'kitchens',
       'owner_name', 'bedrooms', 'ac_type', 'parking_spots', 'foundation',
       'year'],
      dtype='object')
28
Index(['sale_date', 'sale_price', 'parcel', 'street_no', 'living_area',
       'category', 'price_per_sf', 'street_name', 'y', 'full_address', 'x',
       'owner_property_count', 'total_room_num', 'half_bathrooms',
       'fireplaces', 'bathrooms', 'exterior_condition', 'year_built',
       'properties', 'interior_condition', 'heat_type', 'kitchens',
       'owner_name', 'bedrooms', 'ac_type', 'parking_spots', 'foundation',
       'year'],
      dtype='object')
28
Index(['sale_date', 'sale_price', 'parcel', 'street_no', 'l

In [44]:
print(dataframes)

[       sale_date  sale_price        parcel  street_no  living_area  \
0      6/20/2019    725000.0  01-02114-000       56.0       1532.0   
1      6/27/2019    510000.0  01-06784-000      177.0        976.0   
2       3/8/2019    650000.0  01-02831-000       73.0       1878.0   
3      6/21/2019    445000.0  01-04931-000       15.0        850.0   
4       6/4/2019    725200.0  01-05187-010      172.0       1512.0   
...          ...         ...           ...        ...          ...   
6194   12/4/2019    685000.0  22-03256-000       27.0       2432.0   
6195    1/8/2019   1460000.0  22-05360-000      226.0       2880.0   
6196   7/10/2019   1062000.0  22-03385-000      673.0       3145.5   
6197   10/4/2019    900000.0  22-00618-000      474.0       2250.0   
6198  10/18/2019    900000.0  01-04069-000      125.0       3684.0   

           category  price_per_sf    street_name          y  \
0     single_family        473.24   BEACHVIEW RD  42.391103   
1     single_family        522.5

In [45]:
# merge all the dataframes
data = pd.concat(dataframes)

# save the merged dataframe
data.to_csv("merged_geocoded_enriched_merged_sales.csv", index=False)



In [46]:
data[data["x"].isna() | data["y"].isna()]

,sale_date,sale_price,parcel,street_no,living_area,category,price_per_sf,street_name,y,full_address,...,properties,interior_condition,heat_type,kitchens,owner_name,bedrooms,ac_type,parking_spots,foundation,year


In [47]:
# check rows that have None for bedrooms
data[data["bedrooms"].isna() | data["living_area"].isna()][["bedrooms", "living_area"]]


,bedrooms,living_area
13,NaN,1580.0
15,NaN,1145.0
17,NaN,1152.0
161,NaN,2488.0
227,NaN,2085.0
...,...,...
1028,NaN,3402.0
1077,NaN,4128.0
1194,NaN,3076.0
6092,NaN,3435.0


In [48]:
data["living_area"]

0       1532.0
1        976.0
2       1878.0
3        850.0
4       1512.0
         ...  
4952    2814.0
4953    2325.0
4954    2208.0
4955    2268.0
4956    2506.0
Name: living_area, Length: 31227, dtype: float64

In [49]:
data[data["living_area"].isna()]

,sale_date,sale_price,parcel,street_no,living_area,category,price_per_sf,street_name,y,full_address,...,properties,interior_condition,heat_type,kitchens,owner_name,bedrooms,ac_type,parking_spots,foundation,year


In [50]:
data[data["sale_price"].isna()]

,sale_date,sale_price,parcel,street_no,living_area,category,price_per_sf,street_name,y,full_address,...,properties,interior_condition,heat_type,kitchens,owner_name,bedrooms,ac_type,parking_spots,foundation,year


In [53]:
# create a new df that has no None for bedrooms 
data_no_na = data[data["bedrooms"].notna()]

print(data_no_na.shape)

(31019, 28)


In [54]:
# Step 1: Calculate the median and MAD
median = data_no_na['sale_price'].median()
mad = np.median(np.abs(data_no_na['sale_price'] - median))

# Step 2: Calculate Modified Z-scores
data_no_na['modified_z'] = 0.6745 * (data_no_na['sale_price'] - median) / mad

# Step 3: Filter out outliers
threshold = 3.5
data_no_na_no_outliers = data_no_na[np.abs(data_no_na['modified_z']) <= threshold].copy()

# check how many rows are removed
print(data_no_na.shape[0] - data_no_na_no_outliers.shape[0])


2539


/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_91298/776181895.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_na['modified_z'] = 0.6745 * (data_no_na['sale_price'] - median) / mad


In [55]:
# Optional: Drop the helper column
data_no_na_no_outliers.drop(columns='modified_z', inplace=True)

print(data_no_na_no_outliers)


       sale_date  sale_price        parcel  street_no  living_area  \
0      6/20/2019    725000.0  01-02114-000       56.0       1532.0   
1      6/27/2019    510000.0  01-06784-000      177.0        976.0   
2       3/8/2019    650000.0  01-02831-000       73.0       1878.0   
3      6/21/2019    445000.0  01-04931-000       15.0        850.0   
4       6/4/2019    725200.0  01-05187-010      172.0       1512.0   
...          ...         ...           ...        ...          ...   
4952  11/27/2023   1000000.0  22-04490-000       12.0       2814.0   
4953   9/28/2023    875000.0  22-03741-000      135.0       2325.0   
4954   4/14/2023    900000.0  22-03853-000       48.0       2208.0   
4955   3/30/2023    875000.0  22-04322-000       96.0       2268.0   
4956  12/21/2023   1095000.0  22-03588-000       40.0       2506.0   

           category  price_per_sf      street_name          y  \
0     single_family        473.24     BEACHVIEW RD  42.391103   
1     single_family        52

In [56]:
# save the new df
data_no_na.to_csv("sales_boston_geocoded.csv", index=False)



In [57]:
data_no_na[data_no_na["sale_price"].isna() | data_no_na["bedrooms"].isna() | data_no_na["living_area"].isna()]

,sale_date,sale_price,parcel,street_no,living_area,category,price_per_sf,street_name,y,full_address,...,interior_condition,heat_type,kitchens,owner_name,bedrooms,ac_type,parking_spots,foundation,year,modified_z
